In [1]:
import plotly.graph_objects as go
import numpy as np
from numpy import cos, sin
import ipywidgets as widgets
from ipywidgets import HBox, VBox
from IPython.display import display, clear_output
from dataclasses import dataclass

## C FFI Test

In [6]:
import ctypes
from ctypes import POINTER, c_double, c_size_t

clibrary = ctypes.CDLL("cpython.so")

clibrary.set_frames_py.argtypes = [
    POINTER(c_double),  # table_x
    POINTER(c_double),  # table_y
    POINTER(c_double),  # table_z
    POINTER(c_double),  # slider_x
    POINTER(c_double),  # slider_y
    POINTER(c_double),  # slider_z
    POINTER(c_double),  # state_A
    POINTER(c_double),  # state_B
    c_size_t            # num_of_frames
]
clibrary.set_frames_py.restype = ctypes.c_bool

state_A = (c_double * 6)(0.0, 0.0, 0.0, -3.1415/10, 3.1415/10, 3.1415/10)
state_B = (c_double * 6)(0.5, 0.5, 3.0, -3.1415/10, 3.1415/10, 3.1415/10)

num_of_frames = 4

table_x = (c_double * (num_of_frames * 8))()  
table_y = (c_double * (num_of_frames * 8))()
table_z = (c_double * (num_of_frames * 8))()

slider_x = (c_double * (num_of_frames * 6))()  
slider_y = (c_double * (num_of_frames * 6))()
slider_z = (c_double * (num_of_frames * 6))()

success = clibrary.set_frames_py(
    table_x, table_y, table_z,
    slider_x, slider_y, slider_z,
    state_A, state_B,
    num_of_frames
)

## Visualisation

In [3]:
AXIS_RANGE = [-15, 15] 
MESH_INDICES = [(0, 1, 6), (2, 3, 6), (4, 5, 6), (1, 4, 6), (3, 0, 6), (5, 2, 6)]
STEP_SIZE = 0.0000001

x_widget_A = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='X Position state A')
y_widget_A = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Y Position state A')
z_widget_A = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Z Position state A')
phi_widget_A = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Phi Angle state A')
theta_widget_A = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Theta Angle state A')
psi_widget_A = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Psi Angle state A')

x_widget_B = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='X Position state B')
y_widget_B = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Y Position state B')
z_widget_B = widgets.FloatSlider(value=0, min=-10, max=10, step=STEP_SIZE, description='Z Position state B')
phi_widget_B = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Phi Angle state B')
theta_widget_B = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Theta Angle state B')
psi_widget_B = widgets.FloatSlider(value=0, min=-np.pi, max=np.pi, step=STEP_SIZE, description='Psi Angle state B')

# homotopy_widget = widgets.FloatSlider(value=0, min=0, max=1, step=STEP_SIZE, description='Homotopy coefficient')

WIDGETS_A = [x_widget_A, y_widget_A, z_widget_A, phi_widget_A, theta_widget_A, psi_widget_A]
WIDGETS_B = [x_widget_B, y_widget_B, z_widget_B, phi_widget_B, theta_widget_B, psi_widget_B]

def find_element_by_tag(fig, tag):
    for i, plot_element in enumerate(fig.data):
        if plot_element.customdata and plot_element.customdata[0] == tag:
            return i
    return None


In [4]:
# def initialize_plot():
#     fig = go.FigureWidget()
#     fig.update_layout(
#         scene=dict(
#             xaxis=dict(range=AXIS_RANGE),
#             yaxis=dict(range=AXIS_RANGE),
#             zaxis=dict(range=AXIS_RANGE)
#         ),
#     title='6-DOF robot with vertical parallel rails')
#     fig.add_scatter3d(x=initial_table_points[:, 0], y=initial_table_points[:, 1], z=initial_table_points[:, 2], mode='markers', marker=dict(size=5), line=dict(color='blue'), name='Initial Table', customdata=["original_table"])
#     fig.add_scatter3d(x=initial_table_points[:, 0], y=initial_table_points[:, 1], z=initial_table_points[:, 2], mode='markers', marker=dict(size=5), line=dict(color='red'), name='Transformed Table', customdata=["transformed_table"])

#     for point in base_points:
#         fig.add_scatter3d(x=[point[0], point[0]], y=[point[1], point[1]], z=[MIN_POSSIBLE_HEIGHT, MAX_POSSIBLE_HEIGHT], mode='lines', line=dict(color='green', width=2), showlegend=False)

#     needle_x = x_widget.value
#     needle_y = y_widget.value
#     needle_z = z_widget.value
#     centroid_x = initial_table_points[6, 0] 
#     centroid_y = initial_table_points[6, 1] 
#     centroid_z = initial_table_points[6, 2] 
#     fig.add_scatter3d(x=[needle_x, centroid_x], y=[needle_y, centroid_y], z=[needle_z, centroid_z], mode='lines', line=dict(color='red', width=2), showlegend=False, customdata=["initial_needle"])

#     fig.add_mesh3d(
#         x=initial_table_points[:, 0], 
#         y=initial_table_points[:, 1], 
#         z=initial_table_points[:, 2], 
#         i=[i[0] for i in MESH_INDICES], 
#         j=[i[1] for i in MESH_INDICES], 
#         k=[i[2] for i in MESH_INDICES], 
#         color='red', 
#         opacity=0.5,
#         customdata=["table_mesh"]
#     )
    
#     for i, (t, b) in enumerate(zip(initial_table_points, base_points)):
#         intersection = find_intersection(t, b)
#         if intersection:
#             _, _, intersection_z = intersection
#             fig.add_scatter3d(x=[t[0], b[0]], y=[t[1], b[1]], z=[t[2], intersection_z], mode='lines', line=dict(color='black', width=2), showlegend=False, customdata=[f"arm_{i}"])

#     fig.update_layout(scene=dict(xaxis=dict(range= AXIS_RANGE, title='X Axis'), yaxis=dict(range= AXIS_RANGE, title='Y Axis'), zaxis=dict(range= AXIS_RANGE, title='Z Axis'), aspectmode='cube'), margin=dict(l=0, r=0, b=0, t=0))
#     return fig

# def update_plot(change):
#     with plot.batch_update():
#         c = NeedleCoordinates(x = x_widget.value, y = y_widget.value, z = z_widget.value, phi = phi_widget.value, theta =theta_widget.value, psi = psi_widget.value)
#         homotopy_coefficient = homotopy_widget.value
#         transformation_matrix = populate_transformation_matrix(c, homotopy_coefficient)
#         transformed_table_points = np.array([transform_vector(point, transformation_matrix) for point in initial_table_points])

#         transformed_table = find_element_by_tag(plot, "transformed_table")
#         mesh = find_element_by_tag(plot, "table_mesh")
#         plot.data[transformed_table].x, plot.data[transformed_table].y, plot.data[transformed_table].z = transformed_table_points.T
#         plot.data[mesh].x, plot.data[mesh].y, plot.data[mesh].z = transformed_table_points.T
#         plot.data[mesh].i, plot.data[mesh].j, plot.data[mesh].k = zip(*MESH_INDICES)


#         needle = find_element_by_tag(plot, "initial_needle")
#         plot.data[needle].x = [transformed_table_points[6][0], transformed_table_points[7][0]]
#         plot.data[needle].y = [transformed_table_points[6][1], transformed_table_points[7][1]]
#         plot.data[needle].z = [transformed_table_points[6][2], transformed_table_points[7][2]]

#         intersection_points = []
#         for i, (t, b) in enumerate(zip(transformed_table_points, base_points)):
#             intersection = find_intersection(t, b)
#             arm = find_element_by_tag(plot, f"arm_{i}")

#             if intersection:
#                 intersection_points.append(intersection)
#                 _, _, intersection_z = intersection
#                 plot.data[arm].x = [t[0], b[0]]
#                 plot.data[arm].y = [t[1], b[1]]
#                 plot.data[arm].z = [t[2], intersection_z]
#             else:
#                 intersection_points.append(None)
#                 plot.data[arm].x = [None, None]
#                 plot.data[arm].y = [None, None]
#                 plot.data[arm].z = [None, None]

#         intersection_text = format_intersection_points(intersection_points)
#         text_widget.value = intersection_text

In [5]:
base_points = np.array([
    [7.0 * np.sin(np.deg2rad(60) * i + np.deg2rad(15) / 2), 7.0 * np.cos(np.deg2rad(60) * i + np.deg2rad(15) / 2), 0] for i in range(6)
])

def initialize_plot():
    fig = go.FigureWidget()
    fig.update_layout(
        scene=dict(xaxis=dict(range=AXIS_RANGE), yaxis=dict(range=AXIS_RANGE), zaxis=dict(range=AXIS_RANGE)),
        title='6-DOF robot with vertical parallel rails'
    )
    
    fig.add_scatter3d(mode='markers', marker=dict(size=5), line=dict(color='blue'), name='Initial Table Points')
    fig.add_scatter3d(mode='markers', marker=dict(size=5), line=dict(color='red'), name='Final Table Points')
    fig.add_mesh3d(color='blue', opacity=0.5, name='Initial Table Mesh')
    fig.add_mesh3d(color='red', opacity=0.5, name='Final Table Mesh')

    fig.update_layout(scene=dict(xaxis=dict(range=AXIS_RANGE, title='X Axis'), yaxis=dict(range=AXIS_RANGE, title='Y Axis'), zaxis=dict(range=AXIS_RANGE, title='Z Axis'), aspectmode='cube'), margin=dict(l=0, r=0, b=0, t=0))
    return fig

def update_plot(change):
    state_A = (ctypes.c_double * 6)(x_widget_A.value, y_widget_A.value, z_widget_A.value, phi_widget_A.value, theta_widget_A.value, psi_widget_A.value)
    state_B = (ctypes.c_double * 6)(x_widget_B.value, y_widget_B.value, z_widget_B.value, phi_widget_B.value, theta_widget_B.value, psi_widget_B.value)
    num_of_frames = 2
    
    # Prepare arrays to hold the result
    table_x = (ctypes.c_double * (num_of_frames * 8))()
    table_y = (ctypes.c_double * (num_of_frames * 8))()
    table_z = (ctypes.c_double * (num_of_frames * 8))()
    slider_x = (ctypes.c_double * (num_of_frames * 6))()
    slider_y = (ctypes.c_double * (num_of_frames * 6))()
    slider_z = (ctypes.c_double * (num_of_frames * 6))()

    # Call the C function
    success = clibrary.set_frames_py(
        table_x, table_y, table_z,
        slider_x, slider_y, slider_z,
        state_A, state_B,
        num_of_frames
    )

    if success:
        initial_frame_points = np.array([[table_x[i], table_y[i], table_z[i]] for i in range(8)])
        final_frame_points = np.array([[table_x[8 + i], table_y[8 + i], table_z[8 + i]] for i in range(8)])

        with plot.batch_update():
            # Update the initial table points (frame 0)
            plot.data[0].x, plot.data[0].y, plot.data[0].z = initial_frame_points.T

            # Update the final table points (frame 1)
            plot.data[1].x, plot.data[1].y, plot.data[1].z = final_frame_points.T

            # Update the mesh for the initial frame
            plot.data[2].x, plot.data[2].y, plot.data[2].z = initial_frame_points.T
            plot.data[2].i = [i[0] for i in MESH_INDICES]
            plot.data[2].j = [i[1] for i in MESH_INDICES]
            plot.data[2].k = [i[2] for i in MESH_INDICES]

            # Update the mesh for the final frame
            plot.data[3].x, plot.data[3].y, plot.data[3].z = final_frame_points.T
            plot.data[3].i = [i[0] for i in MESH_INDICES]
            plot.data[3].j = [i[1] for i in MESH_INDICES]
            plot.data[3].k = [i[2] for i in MESH_INDICES]

plot = initialize_plot()

for widget in WIDGETS_A + WIDGETS_B:
    widget.observe(update_plot, names='value')

# Initial trigger
update_plot(None)

# Display widgets
display(HBox([VBox(WIDGETS_A), VBox(WIDGETS_B)]))
display(plot)

FigureWidget({
    'data': [{'line': {'color': 'blue'},
              'marker': {'size': 5},
              'mode': 'markers',
              'name': 'Initial Table Points',
              'type': 'scatter3d',
              'uid': '333ef323-55d6-453c-93dd-675c9ef391f7',
              'x': array([ 1.92836283, -1.92836283,  1.02606043,  2.95442326, -2.95442326,
                          -1.02606043,  0.        ,  0.        ]),
              'y': array([ 2.29813333,  2.29813333, -2.81907786,  0.52094453,  0.52094453,
                          -2.81907786,  0.        ,  0.        ]),
              'z': array([3., 3., 3., 3., 3., 3., 3., 0.])},
             {'line': {'color': 'red'},
              'marker': {'size': 5},
              'mode': 'markers',
              'name': 'Final Table Points',
              'type': 'scatter3d',
              'uid': '34bcfef3-5e02-4c31-9140-89e0708f8d96',
              'x': array([ 1.92836283, -1.92836283,  1.02606043,  2.95442326, -2.95442326,
              